# Bring Your Own Checkpoint (BYOC) to SamabaStudio 

Example checkpoint download from Huggingface

In [ ]:
import os
from huggingface_hub import hf_hub_download, HfApi

model_name = "NousResearch/Hermes-2-Theta-Llama-3-8B"  # Replace with your desired model

target_dir = "./models"  
os.makedirs(target_dir, exist_ok=True)

repo_files = HfApi().list_repo_files(model_name)

for file_name in repo_files:
    hf_hub_download(repo_id=model_name, filename=file_name, cache_dir=target_dir)

After download, modify [config.yaml](./config.yaml) file with the snapshot of the model and required parameters

check at the `chat_template` key in the [tokenizer_config.json](./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725/tokenizer_config.json) file inside the model snapshot and modify it to follow a chat jinja template like this:

```json
  "chat_template": "{% for message in messages %}{% set content = '<|im_start|>' + message['role'] + '\n' + message['content'] | trim + '<|im_end|>'+'\n' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}<|im_start|>assistant\n"
```

In [2]:
from src.snsdk_byoc_wrapper import BYOC
byoc = BYOC("./config.yaml")

2024-11-19 11:22:21,683 [INFO] Using config file located in ./config.yaml
2024-11-19 11:22:21,686 [INFO] Using variables from Snapi config to set up Snsdk.


Check your jinja chat template

In [3]:
test_messages = [
    {"role": "system", "content": "This is a system prompt."},
    {"role": "user", "content": "This is a user prompt."},
    {"role": "assistant", "content": "This is a response from the assistant."},
    {"role": "user", "content": "This is an user follow up"}
    ]
byoc.check_chat_templates(test_messages)

2024-11-19 11:03:09,282 [INFO] Raw chat template for checkpoint in ./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725:
{% for message in messages %}{% set content = '<|im_start|>' + message['role'] + '
' + message['content'] | trim + '<|im_end|>'+'
' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}<|im_start|>assistant


2024-11-19 11:03:09,289 [INFO] Rendered template with input test messages:

<|begin_of_text|><|im_start|>system
This is a system prompt.<|im_end|>
<|im_start|>user
This is a user prompt.<|im_end|>
<|im_start|>assistant
This is a response from the assistant.<|im_end|>
<|im_start|>user
This is an user follow up<|im_end|>
<|im_start|>assistant


Set BYOC object with checkpoints to upload config

Get params from checkpoints paths set in [config.yaml](./config.yaml) file

In [4]:
byoc.find_config_params(update_config_file=True)

2024-11-18 14:45:09,453 [INFO] Params for checkpoint in ./models/models--NousResearch--Hermes-2-Theta-Llama-3-8B/snapshots/57a73110702e7b05ba3f39fef36297454c680725:
[{'model_arch': 'llama', 'seq_length': 8192, 'vocab_size': 128256}]
2024-11-18 14:45:09,455 [INFO] config updated with checkpoints parameters
2024-11-18 14:45:09,457 [INFO] config file updated with checkpoints parameters


Find suitable SambaStudio apps for your checkpoints

In [2]:
byoc.get_suitable_apps()

2024-11-18 20:15:00,196 [INFO] Checkpoint HermesProInstructV10 suitable apps:
{'id': '61fa0993-04a2-42ca-9db1-1eff693ea978', 'name': 'Samba1 Llama3 Experts'}
{'id': 'ad39e323-9878-4914-8e29-82c9f2939475', 'name': 'Llama 3'}


Set in the `app_id` key of your checkpoint in the [config.yaml](./config.yaml) file the desired app_id of your checkpoints

Upload the checkpoints in your [config.yaml](./config.yaml) file to SambaStudio (this could take from minutes to hours depending on the size of your checkpoints)

In [3]:
#load the updated config.yaml file including app ids and model params
byoc = BYOC("./config.yaml")

2024-11-19 11:22:25,609 [INFO] Using config file located in ./config.yaml
2024-11-19 11:22:25,610 [INFO] Using variables from Snapi config to set up Snsdk.


In [ ]:
byoc.upload_checkpoints()

You can check the status of the checkpoints after the upload process

In [10]:
byoc.get_checkpoints_status()

2024-11-18 22:01:49,150 [INFO] model HermesProInstructV10 status: 
 {'model_id': '0402d3fd-3ad1-4368-837b-5323bcac842c', 'status': 'Available', 'progress': 100, 'stage': 'convert', 'status_code': 200}


We can create a bundle model including uploaded and existing checkpoints in sambastudio setting `composite_model` params in [config.yaml](./config.yaml) file

In [11]:
byoc.create_composite_model()

2024-11-18 22:01:51,024 [INFO] Model with name 'TestByocCoE' found with id 6aada915-b864-489f-8663-22331ea5d457
2024-11-18 22:01:51,024 [INFO] Model with name 'TestByocCoE' not created it already exist with id 6aada915-b864-489f-8663-22331ea5d457


'6aada915-b864-489f-8663-22331ea5d457'

Then we can deploy the bundle model, setting `deployement` params in [config.yaml](./config.yaml) file

In [12]:
# create project
byoc.create_project()

2024-11-18 22:01:55,219 [INFO] Project with name 'example-byoc-project' found with id 05872601-c99b-4ece-9ccf-381821a1a9a5
2024-11-18 22:01:55,220 [INFO] Project with name 'example-byoc-project' already exists with id '05872601-c99b-4ece-9ccf-381821a1a9a5', using it


'05872601-c99b-4ece-9ccf-381821a1a9a5'

In [13]:
# deploy your model in an endpoint
byoc.create_endpoint()

2024-11-18 22:01:56,619 [INFO] Project with name 'example-byoc-project' found with id 05872601-c99b-4ece-9ccf-381821a1a9a5
2024-11-18 22:01:56,869 [INFO] Model with name 'TestByocCoE' found with id 6aada915-b864-489f-8663-22331ea5d457
2024-11-18 22:01:57,153 [INFO] Endpoint with name 'test-endpoint-byoc' not found in project '05872601-c99b-4ece-9ccf-381821a1a9a5'
2024-11-18 22:01:57,468 [INFO] Endpoint 'test-endpoint-byoc' created


'8839d682-278b-4d09-8eaf-f93cdf5963aa'

In [ ]:
# get endpoint details
byoc.get_endpoint_details()

Use the models

In [11]:
from langchain_community.chat_models import ChatSambaStudio

endpoint_details = byoc.get_endpoint_details()
endpoint_url = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_URL")
endpoint_api_key = endpoint_details["langchain_wrapper_env"].get("SAMBASTUDIO_API_KEY")

llm = ChatSambaStudio(
    sambastudio_url=endpoint_url,
    sambastudio_api_key=endpoint_api_key,
    do_sample= True, 
    temperature= 0.01,
    max_tokens= 512,
    top_p= 1.0,
    top_k= 50,
    process_prompt=False,
    model= "HermesProInstructV10"
)
print(llm.invoke("tell me a joke").content)

2024-11-05 09:31:36,059 [INFO] Project with name 'example-byoc-project' found with id 9d17a0b5-b0f1-478c-9edb-51e8c93fd492


Here's one:

Why don't scientists trust atoms?

Because they make up everything!
